In [16]:
import os
from dotenv import load_dotenv
import requests
import json
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

In [17]:
load_dotenv()

TMDB_API_KEY_TV_SERIES = os.getenv("TMDB_API_KEY_TV_SERIES")

INPUT_PATH = "../../data/id_list/tv_series_ids_12_31_2025.jsonl"
OUTPUT_PATH = "data/tv_series/tv_series_12_31_2025.jsonl"
MAX_WORKERS = 5
CHUNK_SIZE=25

print(TMDB_API_KEY_TV_SERIES)

b98079a3b08cb6c409702b802a572d53


In [18]:
url_tv_series_detail = f"https://api.themoviedb.org/3/tv"
headers_detail = {
    "Accept": "application/json"
}
params_details = {
    'api_key': TMDB_API_KEY_TV_SERIES,
    'language': 'en-US',
    'append_to_response': 'videos,images'
}


In [19]:
def get_tv_series_data(tv_series_id):

    tv_series_resp = requests.get(f"{url_tv_series_detail}/{tv_series_id}", headers=headers_detail, params=params_details)

    credit_resp = requests.get(f"{url_tv_series_detail}/{tv_series_id}/credits", headers=headers_detail, params=params_details)
    credit_info =  credit_resp.json()

    cast_list = []
    if credit_info.get("cast") is None:
        cast_list = []
        print("No cast found")
    else:
        cast_info = credit_info["cast"]
        for info in cast_info:
            cast = {
                "person_id": info.get("id"),
                "known_for_department": info.get("known_for_department"),
                "cast_id": info.get("cast_id"),
                "character": info.get("character"),
                "credit_id": info.get("credit_id"),
            }
            cast_list.append(cast)

    crew_list = []
    if credit_info.get("crew") is None:
        cast_list = []
        print("No crew found")
    else:
        crew_info = credit_info["crew"]
        for info in crew_info:
            crew = {
                "person_id": info.get("id"),
                "known_for_department": info.get("known_for_department"),
                "department": info.get("department"),
                "job": info.get("job"),
            }
            crew_list.append(crew)

    final_resp = {
        "tv_series_id": tv_series_id,
        "tv_series_detail": tv_series_resp.json(),
        "casts_info": cast_list,
        "crews_info": crew_list,
    }

    return final_resp

def count_lines(file_path):
    with open(file_path, "rb") as f:
        return sum(1 for _ in f)

In [20]:
reader = pd.read_json(INPUT_PATH, lines=True, chunksize=CHUNK_SIZE)
total_lines = count_lines(INPUT_PATH)
pbar = tqdm(total=total_lines, desc="Sedding tv series data", unit=" tv series")

idx_line = 0
for chunk in reader:
    batch_list = chunk.to_dict(orient="records")
    tv_series_ids = [item["id"] for item in batch_list]
    results = []

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = []

        for tv_series_id in tv_series_ids:
            futures.append(executor.submit(get_tv_series_data, tv_series_id))

        for future in futures:
            try:
                tv_series_data = future.result()
                if tv_series_data:
                    results.append(tv_series_data)
            except Exception as e:
                pbar.write(f"Error in thread: {e}")
            finally:
                pbar.set_postfix({"idx_in_file": idx_line}, refresh=False)
                pbar.update(1)
                idx_line += 1

    # for id, item in enumerate(batch_list):
    #     pbar.set_postfix({"processing_tv_series_id": item["id"], "idx_in_file": idx_line}, refresh=False)
    #     try:
    #         tv_series_data = get_tv_series_data(item["id"])
    #     except Exception as e:
    #         pbar.write(f"Error processing tv_series {item['id']}: {e}")
    #     finally:
    #         pbar.update(1)
    #         idx_line += 1
    #
    #     results.append(tv_series_data)

    with open(OUTPUT_PATH, "a", encoding="utf-8") as f:
        for res in results:
            f.write(json.dumps(res) + "\n")

pbar.close()
print("Done!")


Sedding tv series data:   4%|▍         | 8221/211851 [38:46<17:50:42,  3.17 tv series/s, idx_in_file=8220]

No cast found
No crew found



Sedding tv series data:   4%|▍         | 8930/211851 [42:05<11:54:31,  4.73 tv series/s, idx_in_file=8929]

No cast found
No crew found



Sedding tv series data:   5%|▍         | 9731/211851 [45:50<14:28:35,  3.88 tv series/s, idx_in_file=9730]

KeyboardInterrupt: 